In [20]:
import subprocess
import shlex
from pathlib import Path
from typing import Sequence, Union

class RifeADAPTER:
    def __init__(
        self,
        img: Sequence[Union[str, Path]],
        exp: int = 4,
        ratio: float = 0.0,
        rthreshold: float = 0.02,
        rmaxcyc: int = 8,
        model: Union[str, Path] = "ECCV2022-RIFE/train_log",
        out_dir: Union[str, Path] = "./output"
    ):
        self.img = [str(Path(p)) for p in img]
        self.exp = int(exp)
        self.ratio = float(ratio) # inference ratio between two images with 0 - 1 range
        self.rthreshold = float(rthreshold) # returns image when actual ratio falls in given range threshold
        self.rmaxcyc = int(rmaxcyc) # limit max number of bisectional cycles
        self.model = str(model) # directory with trained model files
        self.out_dir = str(out_dir)

    def _build_cmd(self) -> list[str]:
        return [
            "python3", "ECCV2022-RIFE/inference_img.py",
            "--img", *self.img,
            "--exp", str(self.exp),
            "--ratio", str(self.ratio),
            "--rthreshold", str(self.rthreshold),
            "--rmaxcycles", str(self.rmaxcyc),
            "--model", self.model,
        ]

    def _build_ffmpeg_cmd(
        self,
        pattern: str,
        gif_path: str,
        framerate: int,
        size: str,
    ) -> list[str]:
        return [
            "ffmpeg",
            "-y",                       # перезаписывать без подтверждения
            "-r", str(framerate),
            "-f", "image2",
            "-i", pattern,              # например output/img%d.png
            "-s", size,   
            "-vf",
            "split[s0][s1];[s0]palettegen=stats_mode=single[p];"
            "[s1][p]paletteuse=new=1",
            gif_path,
        ]
        
    def __call__(self, **subprocess_kwargs):
        cmd = self._build_cmd()
        print("Executing:", shlex.join(cmd))
        return subprocess.run(cmd, check=True, **subprocess_kwargs)


    def create_gif(
        self,
        framerate: int = 10,
        size: str = "448x256",
        pattern: str | None = None,
        gif_name: str = "slomo.gif",
        **subprocess_kwargs,
    ):
        pattern = pattern or f"{self.out_dir}/img%d.png"
        gif_path = gif_name if Path(gif_name).is_absolute() else f"{self.out_dir}/{gif_name}"
        Path(gif_path).parent.mkdir(parents=True, exist_ok=True)

        cmd = self._build_ffmpeg_cmd(pattern, gif_path, framerate, size)
        print("Executing:", shlex.join(cmd))
        return subprocess.run(cmd, check=True, **subprocess_kwargs)

In [21]:
adapter = RifeADAPTER(img=['ECCV2022-RIFE/demo/I0_0.png', 'ECCV2022-RIFE/demo/I0_1.png'])
result = adapter(capture_output=True, text=True)
print(result.stdout)

Executing: python3 ECCV2022-RIFE/inference_img.py --img ECCV2022-RIFE/demo/I0_0.png ECCV2022-RIFE/demo/I0_1.png --exp 4 --ratio 0.0 --rthreshold 0.02 --rmaxcycles 8 --model ECCV2022-RIFE/train_log
Loaded v3.x HD model.



In [22]:
adapter.create_gif(framerate=12, size="512x288") 

Executing: ffmpeg -y -r 12 -f image2 -i ./output/img%d.png -s 512x288 -vf 'split[s0][s1];[s0]palettegen=stats_mode=single[p];[s1][p]paletteuse=new=1' ./output/slomo.gif


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

CompletedProcess(args=['ffmpeg', '-y', '-r', '12', '-f', 'image2', '-i', './output/img%d.png', '-s', '512x288', '-vf', 'split[s0][s1];[s0]palettegen=stats_mode=single[p];[s1][p]paletteuse=new=1', './output/slomo.gif'], returncode=0)

In [25]:
import subprocess
import shlex
from pathlib import Path
from typing import Union


class tpsAdapter:
    def __init__(
        self,
        image1: Union[str, Path] = "tps-inbetween/assets/input1_0.png",
        image2: Union[str, Path] = "tps-inbetween/assets/input1_1.png",
        xN: int = 30,
        save_path: Union[str, Path] = "./output",
        cpu: bool = False,
        model_path: Union[str, Path] = "tps-inbetween/ckpt/model_latest.pt",
    ):
        self.image1 = str(Path(image1))
        self.image2 = str(Path(image2))
        self.xN = int(xN)
        self.save_path = str(Path(save_path))
        self.cpu = bool(cpu)
        self.model_path = str(Path(model_path))

    def _build_cmd(self) -> list[str]:
        cmd = [
            "python", "tps-inbetween/demo.py",
            "--image1", self.image1,
            "--image2", self.image2,
            "--xN", str(self.xN),
            "--save_path", self.save_path,
            "--model_path", self.model_path,
        ]
        if self.cpu:
            cmd.append("--cpu")
        return cmd

    def __call__(self, **subprocess_kwargs):
        cmd = self._build_cmd()
        print("Executing:", shlex.join(cmd))
        return subprocess.run(cmd, check=True, **subprocess_kwargs)


In [27]:
adapter = tpsAdapter(
    image1='tps-inbetween/assets/input1_0.png',
    image2='tps-inbetween/assets/input1_1.png',
    xN=30,
    save_path='./output',
    cpu=False,                                   
    model_path='tps-inbetween/ckpt/model_latest.pt',
)

result = adapter(capture_output=True, text=True)
print(result.stdout)  

Executing: python tps-inbetween/demo.py --image1 tps-inbetween/assets/input1_0.png --image2 tps-inbetween/assets/input1_1.png --xN 30 --save_path output --model_path tps-inbetween/ckpt/model_latest.pt
======= Start Loading Matching Model======
Successfully loading matching model! Start matching ...
Matching complete! Results have saved to  output/matching.png
====== Start Inbetweening ======
Finish! Results have saved to  output/out.gif



In [6]:
import subprocess
import shlex
from pathlib import Path
from typing import Union


class sainAdapter:
    def __init__(
        self,
        frameA: Union[str, Path],
        frameB: Union[str, Path],
        out: Union[str, Path] = "SAIN/inbetween.png",
        ckpt: Union[str, Path] = "SAIN/checkpoint/model_best.pth",
        size: int = 512,
        device: str = "cuda",
    ):
        if size % 8 != 0:
            raise ValueError("--size должно быть кратно 8")

        self.frameA = str(Path(frameA))
        self.frameB = str(Path(frameB))
        self.out = str(Path(out))
        self.ckpt = str(Path(ckpt))
        self.size = int(size)
        self.device = device.lower()  # 'cuda' или 'cpu'

    def _build_cmd(self) -> list[str]:
        return [
            "python", "SAIN/infer.py",
            "--frameA", self.frameA,
            "--frameB", self.frameB,
            "--out", self.out,
            "--ckpt", self.ckpt,
            "--size", str(self.size),
            "--device", self.device,
        ]

    def __call__(self, **subprocess_kwargs):
        cmd = self._build_cmd()
        print("Executing:", shlex.join(cmd))
        return subprocess.run(cmd, check=True, **subprocess_kwargs)


In [7]:
adapter = sainAdapter(
    frameA="SAIN/frame1.png",
    frameB="SAIN/frame2.png",
    out="result.png",
    size=512,           
    device="cuda"       
)

adapter()

Executing: python SAIN/infer.py --frameA SAIN/frame1.png --frameB SAIN/frame2.png --out result.png --ckpt SAIN/checkpoint/model_best.pth --size 512 --device cuda


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


✓ saved → result.png


CompletedProcess(args=['python', 'SAIN/infer.py', '--frameA', 'SAIN/frame1.png', '--frameB', 'SAIN/frame2.png', '--out', 'result.png', '--ckpt', 'SAIN/checkpoint/model_best.pth', '--size', '512', '--device', 'cuda'], returncode=0)

In [1]:
from rife import RifeAdapter
from tps  import TpsAdapter
from sain import SainAdapter


ADAPTERS = {
    "rife": RifeAdapter(),    
    "tps" : TpsAdapter(),
    "sain": SainAdapter(),
}

def run_all(img1: str, img2: str, *, verbose=True, **subproc_kw):
    results = {}
    for name, adapter in ADAPTERS.items():
        if verbose:
            print(f"\n=== {name.upper()} ===")
        results[name] = adapter(img1, img2, **subproc_kw)
    return results

In [2]:
imgA = "SAIN/frame1.png"
imgB = "SAIN/frame2.png"

logs = run_all(imgA, imgB, capture_output=True, text=True)

for name, proc in logs.items():
    print(f"\n--- {name} stdout ---")
    print(proc.stdout)


=== RIFE ===
Executing: python3 ECCV2022-RIFE/inference_img.py --img SAIN/frame1.png SAIN/frame2.png --exp 4 --ratio 0.0 --rthreshold 0.02 --rmaxcycles 8 --model ECCV2022-RIFE/train_log
Loaded v3.x HD model.

=== TPS ===
Executing: python tps-inbetween/demo.py --image1 SAIN/frame1.png --image2 SAIN/frame2.png --xN 30 --save_path ./output --model_path tps-inbetween/ckpt/model_latest.pt


/workspace/ADAPTERS/tps-inbetween/model/gluestick/models/gluestick.py:346: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


======= Start Loading Matching Model======
Successfully loading matching model! Start matching ...
Matching complete! Results have saved to  ./output/matching.png
====== Start Inbetweening ======
Finish! Results have saved to  ./output/out.gif

=== SAIN ===
Executing: python SAIN/infer.py --frameA SAIN/frame1.png --frameB SAIN/frame2.png --out output/result_frame1_frame2_sain.png --ckpt SAIN/checkpoint/model_best.pth --size 512 --device cuda


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


✓ saved → output/result_frame1_frame2_sain.png

--- rife stdout ---
None

--- tps stdout ---
None

--- sain stdout ---
None
